<a href="https://colab.research.google.com/github/os-angel/TFT/blob/main/TFT_Medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Temporal Fusion Transformers**

Es una arquitectura Deel Learning muy pontente, generalmente usada cuando se quiere capturar dependencias a largo plazo e incluir la influiencia de covariables en el pasado y en el futuro y agregarle interpretabilidad al modelo creado. Generalemte dan buenos resultados comparado con estructuras como ARIMA, RNN o LSTM.

El objetivo de este tutorial es:

* Entender la arquitectura de un Temporal Fusion Transformer.
* Implementar un TFT desde cero.
* Ajustar el modelo TFT mediante Fine Tuning a un caso específico.


### **Orden lógico del tutorial**
Este notebook está ordenado por 11 pasos fundamentales para la implementación de un TFT, en el siguiente orden:

1. Carga y preparación de los datos
2. Preprocesamiento
3. Definición del modelo TFT
4. Entrenamiento y evaluación
5. Predicción y visualización
6. Interpretabilidad del modelo

### **1. Carga y preparación de los datos**

Es importante instalar la librería darts y que el entorjo de ejecución sea a través de la GPU para poder paralelizar las ejecuciones y que sea todo muy eficiente.

In [19]:
# Instalamos la librería dats
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 78.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.0
    Uninstalling pandas-2.0.0:
      Successfully uninstalled pandas-2.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
# Importamos las librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
from darts.models import TFTModel
from darts import TimeSeries
from darts.metrics import rmse
from darts.explainability import TFTExplainer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from google.colab import drive
from darts.dataprocessing.transformers import Scaler
import itertools
import torch


### **2. Definición del modelo TFT**
Para aplicar el modelo, es necesario conocer algunos parámetros importantes y cómo afectan en el rendimiento del mismo:

* **Input Chunk Length:** permite seleccionar el tamaño de la data histórica que se usa cono entrada en el modelo. Este parámetro es importante para darle el mayor contexto posible al modelo y que pueda generar pronósticos más precisos.

* **Output Chunk Length:** indica la cantidad de predicciones u horizonte de predicciones se deben realizar ($\tau$), por ejemplo, si colocamos 24 en este modelo, representa 24 predicciones en la ventana de tiempo establecida.

* **Hidden size:** representa la cantidad de unidades ocultas dentro de la capa del LSTM, este parámetro influye en la capacidad de aprender patrones complejos. A medida que aumentamos la cantidad de unidades ocultas, también aumenta el coste computacional para procesar la información.

* **LSTM Layers:** determina el número de capas LSTM en el modelo. Por ejemplo, si colocamos 2 capas, el modelo puede aprender mejor una estructura de datos jerárquicos (long-term dependencies). Añadir más capas pueden mejorar la precisión pero también incremente la complejidad del modelo y el tiempo de entrenamiento.

* **Number of Attention Heads:** configura la cantidad de attention heads en el mecanismo Multi-head-Attention. Este parámetro mejora la capacidad del modelo de aprender y mejora la precisión en el pronóstico.

* **Dropout Rate:** es una técnica que permite el overfitting mediante una desactivación de algunas unidades de forma aleatoria durante el entrenamiento, ayudando a generalizar mejor ante nuevos datos.

* **Batch Size:** determina cuántas muestras puede manejar el modelo antes de actualizar sus pesos. p.ej. un batch size de 64 indica que el modelo puede procesar 64 muestras juntas. También es importante tomar en cuenta que este parámetro balancea la velocidad de entrenamiento y el uso de memoria para asegurar la eficiencia de entrenamiento.

* **Number of epochs:** Determina cuántas veces el modelo necesita ver el dataset de entrenamiento (aunque es de forma aleatoria), un valor de 10 significa que el modelo iterará sobre el dataset 10 veces. A medida que aumentamos los epochs, el performance del modelo lo hace también pero debemos tener cuidado de no caer en overfitting.

* **Static Covariates**: Este ajuste nos permite incluir variables estáticas, es decir, que permanecen constantes a medida que pasa el tiempo.

In [ ]:
# Cargamos los datos a analizar
df = pd.read_csv('/content/drive/MyDrive/Datasets/electricity.csv', index_col='ds', parse_dates=True)
series = TimeSeries.from_dataframe(df, value_cols='y', fill_missing_dates=True, freq='H')
# Cargamos covariables pasadas
X_past = df[['Exogenous1', 'Exogenous2']]
covariates = TimeSeries.from_dataframe(X_past, fill_missing_dates=True, freq='H')

# Cargamos covariables futuras
future_df = pd.read_csv('/content/drive/MyDrive/Datasets/electricity-future.csv', index_col='ds', parse_dates=True)
X_future = future_df[['Exogenous1', 'Exogenous2']]
X = pd.concat([X_past, X_future])
future_covariates = TimeSeries.from_dataframe(X, fill_missing_dates=True, freq='H')

In [ ]:
# Normalizamos los datos
scaler1, scaler2 = Scaler(), Scaler()
y_transformed = scaler1.fit_transform(series)
past_covariates_transformed = scaler2.fit_transform(covariates)
future_covariates_transformed = scaler2.transform(future_covariates)

In [ ]:
# Creamos una función para Optuna
def objective(trial):
    hidden_size = trial.suggest_int("hidden_size", 16, 128)
    lstm_layers = trial.suggest_int("lstm_layers", 1, 4)
    num_attention_heads = trial.suggest_int("num_attention_heads", 2, 8)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    n_epochs = trial.suggest_int("n_epochs", 10, 50)

    # Creamos el TFT con hiperparámetros a optimizar con Optuna
    model = TFTModel(
        input_chunk_length=96,
        output_chunk_length=24,
        hidden_size=hidden_size,
        lstm_layers=lstm_layers,
        num_attention_heads=num_attention_heads,
        dropout=dropout,
        batch_size=batch_size,
        n_epochs=n_epochs,
        add_encoders={'cyclic': {'future': ['hour', 'day', 'month']}},
        use_static_covariates=True,
        pl_trainer_kwargs={"accelerator": "gpu" if torch.cuda.is_available() else "cpu", "devices": 1},
    )

    # Entrenamos el modelo
    model.fit(y_transformed, past_covariates_transformed, future_covariates_transformed)

    # Validamos con datos históricos
    cv = model.historical_forecasts(
        y_transformed,
        past_covariates_transformed,
        future_covariates_transformed,
        forecast_horizon=24,
        start=df.shape[0] - 10*24,
        stride=24,
        retrain=True
    )

    # Calculamos RMSE
    rmse_value = np.mean([
        np.sqrt(mean_squared_error(df.y[pred.index], scaler1.inverse_transform(pred).pd_series())) for pred in cv
    ])

    return rmse_value  # Minimizaremos RMSE


In [ ]:
# Ejecutamos Optuna con 50 iteracioes
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Mostramos los mejores hiperparámetros
print(f"Mejor RMSE: {study.best_value}")
print(f"Mejores parámetros: {study.best_params}")

# Entrenamos el modelo con los mejores hiperparámetros
best_params = study.best_params
final_model = TFTModel(
    input_chunk_length=96,
    output_chunk_length=24,
    hidden_size=best_params["hidden_size"],
    lstm_layers=best_params["lstm_layers"],
    num_attention_heads=best_params["num_attention_heads"],
    dropout=best_params["dropout"],
    batch_size=best_params["batch_size"],
    n_epochs=best_params["n_epochs"],
    add_encoders={'cyclic': {'future': ['hour', 'day', 'month']}},
    use_static_covariates=True,
    pl_trainer_kwargs={"accelerator": "gpu" if torch.cuda.is_available() else "cpu", "devices": 1},
)

final_model.fit(y_transformed, past_covariates_transformed, future_covariates_transformed)

In [ ]:
# Guardamos el modelo entrenado
final_model.save("tft_best_model.pth")
print("Modelo guardado como tft_best_model.pth")

# Realizamos las predicciones
forecast = final_model.predict(n=24, past_covariates=past_covariates_transformed, future_covariates=future_covariates_transformed)
forecast = TimeSeries.pd_series(scaler1.inverse_transform(forecast)).rename('TFT')

# Graficamos los resultados
plt.figure(figsize=(15, 6))
plt.plot(df.y['2017-12':], label='Actuals')
plt.plot(forecast, label='Forecast', linestyle='dashed')
plt.xlabel('Time')
plt.ylabel('Electricity Consumption')
plt.title('Forecast Result')
plt.legend()
plt.show()